In [1]:
# Module Importations


In [ ]:
# Project Module Importations

In [1]:
# Pyspark Import and Setup
import findspark
findspark.find()
findspark.init()
import pyspark

In [ ]:
# Constants (i.e. for saving figures)

In [3]:
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

Exception: Java gateway process exited before sending its port number